In [4]:
!cd /content/
!git clone https://github.com/huggingface/diffusers.git

fatal: destination path 'diffusers' already exists and is not an empty directory.


In [5]:
!pip install ./diffusers

Processing ./diffusers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.30.0.dev0-py3-none-any.whl size=2656426 sha256=c61161c2ff7a42278705329cbb3d20a2182102c1a5b0b937624f1ad7fb3a1bf2
  Stored in directory: /tmp/pip-ephem-wheel-cache-456u617d/wheels/95/c5/3b/e1b4269f8a2584de57e75f949a185b48fc4144e9a91fc9965a
Successfully built diffusers
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.30.0.dev0
    Uninstalling diffusers-0.30.0.dev0:
      Successfully uninstalled diffusers-0.30.0.dev0


In [6]:
!pip install -U -r /content/diffusers/examples/text_to_image/requirements.txt

In [7]:
!pip install -q diffusers transformers accelerate peft

In [8]:
!pip install datasets

In [9]:
!accelerate config default

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [10]:
from huggingface_hub import notebook_login
notebook_login()

In [11]:
import os

os.environ['MODEL_NAME'] = f'CompVis/stable-diffusion-v1-2'
os.environ['DATASET_NAME'] = f'AnkitaRPurohit/output_model1'
os.environ['OUTPUT_DIR'] = f'output_model'

In [12]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [13]:
from datasets import load_dataset
dataset = load_dataset("imagefolder",data_dir="/content/drive/MyDrive/data_back/",drop_labels=True)
#dataset = load_dataset("yateronald/processed_demo")

Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
dataset.push_to_hub("AnkitaRPurohit/output_model1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/306 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/AnkitaRPurohit/output_model1/commit/110aa9ecd3397eaf45bce98d270d61c066d00548', commit_message='Upload dataset', commit_description='', oid='110aa9ecd3397eaf45bce98d270d61c066d00548', pr_url=None, pr_revision=None, pr_num=None)

In [14]:
!accelerate launch diffusers/examples/text_to_image/train_text_to_image_lora.py \
--pretrained_model_name_or_path="CompVis/stable-diffusion-v1-2" \
--dataset_name=$DATASET_NAME \
--resolution=512 --center_crop --random_flip \
--train_batch_size=1 \
--gradient_accumulation_steps=4 \
--gradient_checkpointing \
--max_train_steps=5 \
--learning_rate=1e-06 \
--max_grad_norm=1 \
--checkpointing_steps=100000 \
--lr_scheduler="constant" \
--lr_warmup_steps=0 \
--output_dir=$OUTPUT_DIR \
--push_to_hub

2024-08-13 17:29:44.731939: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 17:29:45.141947: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 17:29:45.259279: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-13 17:29:45.928317: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-13 17:29:49.818473: W tensorflow/compiler/tf2

In [15]:
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-2")
pipeline.load_lora_weights("AnkitaRPurohit/output_model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

text_encoder/model.safetensors not found


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

safety_checker/config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-2/snapshots/f519eebf1372598b85ab317ec273a63ebeddd5eb/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-2/snapshots/f519eebf1372598b85ab317ec273a63ebeddd5eb/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-2/snapshots/f519eebf1372598b85ab317ec273a63ebeddd5eb

pytorch_lora_weights.safetensors:   0%|          | 0.00/3.23M [00:00<?, ?B/s]

In [ ]:
image5 = pipeline(prompt=" A high-fashion trench coat with an asymmetrical design, made of reflective material that changes color with the light").images[0]
image5.save("AI_IMG2_.png")

#Image Grid Helper Function from hugging face
def image_grid(imgs,rows,cols):
  assert len(imgs) == rows*cols

  w,h = imgs[0].size
  grid = Image.new('RGB',size=(cols*w,rows*h))
  grid_w,grid_h = grid.size

  for i,img in enumerate(imgs):
    grid.paste(img,box=(i%cols*w,i//cols*h))
  return grid

  all_images = [image5]
  grid = image_grid(all_images,rows=1,cols=4)
  grid

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
image5 = pipeline(prompt=" A casual yet elegant outfit combining denim with silk, featuring subtle embroidery inspired by nature").images[0]
image5.save("AI_IMG2_.png")

#Image Grid Helper Function from hugging face
def image_grid(imgs,rows,cols):
  assert len(imgs) == rows*cols

  w,h = imgs[0].size
  grid = Image.new('RGB',size=(cols*w,rows*h))
  grid_w,grid_h = grid.size

  for i,img in enumerate(imgs):
    grid.paste(img,box=(i%cols*w,i//cols*h))
  return grid

  all_images = [image5]
  grid = image_grid(all_images,rows=1,cols=4)
  grid

  0%|          | 0/50 [00:00<?, ?it/s]